# Setup

In [0]:
%python
from pyspark.sql.functions import col


In [0]:
%python
env = dbutils.secrets.get(scope="CommercialAnalytics", key="env-databricks")
if env:
  db_env = "_" + env
else:
  db_env = ""

In [0]:
%python
spark.sql(f"USE business_revenuemanagement{db_env}.an_revenuemanagement_ods")

# Notes

- testing cicd

# Shared Temp Tables

In [0]:
-- removing PY2 and beyond for now
create or replace temp view cypy_date_map as
SELECT DISTINCT
     CASE
     WHEN DATE_FORMAT(date_day, 'MMdd') = '0229' AND DATE_FORMAT(day_last_year, 'MMdd') = '0228' THEN DATE_SUB(date_day, 1)
     ELSE date_day
     END AS date_day,
     CASE
     WHEN DATE_FORMAT(date_day, 'MMdd') = '0228' AND DATE_FORMAT(day_last_year, 'MMdd') = '0229' THEN DATE_SUB(day_last_year, 1)
     ELSE day_last_year
     END AS day_last_year,
     cypy,
     offset
FROM (SELECT date_day,
          date_day AS day_last_year,
          'CY' AS cypy,
          0 AS offset
     FROM subject_dimensions.ds_dimensions.date_dim
     WHERE date_day BETWEEN DATE_TRUNC('year', ADD_MONTHS(CURRENT_DATE(), -72)) AND DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     UNION ALL
     SELECT ADD_MONTHS(date_day, 12) AS date_day,
          date_day AS day_last_year,
          'PY' AS cypy,
          12 AS offset
     FROM subject_dimensions.ds_dimensions.date_dim
     WHERE date_day BETWEEN DATE_TRUNC('year', ADD_MONTHS(CURRENT_DATE(), -72)) AND DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     AND ADD_MONTHS(date_day, 12) <= DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- UNION ALL
     -- SELECT ADD_MONTHS(date_day, 24) AS date_day,
     --      date_day AS day_last_year,
     --      'PY2' AS cypy,
     --      24 AS offset
     -- FROM subject_dimensions.ds_dimensions.date_dim
     -- WHERE date_day BETWEEN DATE_TRUNC('year', ADD_MONTHS(CURRENT_DATE(), -72)) AND DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- AND ADD_MONTHS(date_day, 24) <= DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- UNION ALL
     -- SELECT ADD_MONTHS(date_day, 36) AS date_day,
     --      date_day AS day_last_year,
     --      'PY3' AS cypy,
     --      36 AS offset
     -- FROM subject_dimensions.ds_dimensions.date_dim
     -- WHERE date_day BETWEEN DATE_TRUNC('year', ADD_MONTHS(CURRENT_DATE(), -72)) AND DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- AND ADD_MONTHS(date_day, 36) <= DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- UNION ALL
     -- SELECT ADD_MONTHS(date_day, 48) AS date_day,
     --      date_day AS day_last_year,
     --      'PY4' AS cypy,
     --      48 AS offset
     -- FROM subject_dimensions.ds_dimensions.date_dim
     -- WHERE date_day BETWEEN DATE_TRUNC('year', ADD_MONTHS(CURRENT_DATE(), -72)) AND DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     -- AND ADD_MONTHS(date_day, 48) <= DATE_SUB(DATE_TRUNC('month', ADD_MONTHS(CURRENT_DATE(), 12)), 1)
     )

In [0]:
/* pulling aag od list from cirium schedule fut hist instead of arc dds */
create or replace temp view aag_ha_od_list as
select distinct od
from business_revenuemanagement_test.an_revenuemanagement_ods.rfm_schd_fut_hist_mth
where carr_group IN ('AS', 'HA')

# Build - OA Only

In [0]:
create or replace temp view snap_shell as
with dd as (
  select
  date_day as dptr_date
from subject_dimensions.ds_dimensions.date_dim
where DATE_DAY BETWEEN '2023-01-01' AND date_add(current_date(), 366)
)
,snap as (
  select
    sd.date_day as snapshot_date,
    dd.dptr_date
  from subject_dimensions.ds_dimensions.date_dim sd
    cross join dd
  where
    date_diff(dd.dptr_date, sd.date_day) <= 366
    and date_diff(dd.dptr_date, sd.date_day) >= 0
)
,snap_mth as (
  select distinct
    snapshot_date,
    trunc(dptr_date, 'month') as dptr_month
  from snap
)
,mkts as (
  select distinct
    'OA' as carr_group,
     m.od
  from subject_revenue.ds_revenue_dw.arc_dds_ticket_coupon_fact a
    join subject_revenue.ds_revenue_dw.market_dim m
      on a.market_key = m.market_key
    join subject_revenue.ds_revenue_dw.carrier_dim oc
      on a.oper_carr_key = oc.carrier_key
    join subject_revenue.ds_revenue_dw.carrier_dim mc
      on a.mktg_carr_key = mc.carrier_key
  where
    CASE
      WHEN oc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD = 'HA' THEN 'HA'
      WHEN mc.ALN_CD IN ('AS', 'QX', 'VX', 'HA') AND oc.ALN_CD IN ('OO') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('HA') THEN 'HA'
      WHEN oc.ALN_CD is null and mc.ALN_CD is null THEN NULL
    ELSE 'OA' END = 'OA'
    and m.od in (select od from aag_ha_od_list)
)
select *
from snap_mth
cross join mkts

In [0]:
create or replace temp view arc_agg as
-- combine greater than or equal to 330 DTD into single snapshot date -- beth is doing this for Aircurve to help clean data
select
    date_add(a.schd_dptr_dt, -330) as snap_dt,
    -- a.schd_dptr_dt as dptr_date,
    trunc(a.schd_dptr_dt, 'month') as dptr_month,
    CASE
      WHEN oc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD = 'HA' THEN 'HA'
      WHEN mc.ALN_CD IN ('AS', 'QX', 'VX', 'HA') AND oc.ALN_CD IN ('OO') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('HA') THEN 'HA'
      WHEN oc.ALN_CD is null and mc.ALN_CD is null THEN NULL
    ELSE 'OA' END AS carr_group,
    m.OD,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.MARKET_MILES_AMT ELSE a.MARKET_MILES_AMT END) AS rpm,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.seg_pax_cnt ELSE a.seg_pax_cnt END) AS pax,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.FARE_AMT ELSE a.FARE_AMT END) AS rev
  from
    subject_revenue.ds_revenue_dw.arc_dds_ticket_coupon_fact a
      join subject_revenue.ds_revenue_dw.arc_transaction_type_dim att
        on a.arc_transaction_type_key = att.arc_transaction_type_key
      join subject_revenue.ds_revenue_dw.market_dim m
        on a.market_key = m.market_key
      join subject_revenue.ds_revenue_dw.carrier_dim oc
        on a.oper_carr_key = oc.carrier_key
      join subject_revenue.ds_revenue_dw.carrier_dim mc
        on a.mktg_carr_key = mc.carrier_key
  where
    a.schd_dptr_dt >= '2023-01-01'
    and a.schd_dptr_dt <= date_add(current_date(), 366)
    and m.od in (select od from aag_ha_od_list) -- only AAG / HA markets
    and CASE
      WHEN oc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD = 'HA' THEN 'HA'
      WHEN mc.ALN_CD IN ('AS', 'QX', 'VX', 'HA') AND oc.ALN_CD IN ('OO') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('HA') THEN 'HA'
      WHEN oc.ALN_CD is null and mc.ALN_CD is null THEN NULL
    ELSE 'OA' END = 'OA'
    and date_diff(a.schd_dptr_dt, a.source_file_dt) >= 330
    and att.arc_transaction_type_cd != 'E' -- robby suggested removing exchanged transactions
  group by all
union all
-- less than 330 DTD
select
    a.source_file_dt as snap_dt,
    -- a.schd_dptr_dt as dptr_date,
    trunc(a.schd_dptr_dt, 'month') as dptr_month,
    CASE
      WHEN oc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD = 'HA' THEN 'HA'
      WHEN mc.ALN_CD IN ('AS', 'QX', 'VX', 'HA') AND oc.ALN_CD IN ('OO') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('HA') THEN 'HA'
      WHEN oc.ALN_CD is null and mc.ALN_CD is null THEN NULL
    ELSE 'OA' END AS carr_group,
    m.OD,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.MARKET_MILES_AMT ELSE a.MARKET_MILES_AMT END) AS rpm,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.seg_pax_cnt ELSE a.seg_pax_cnt END) AS pax,
    SUM(CASE WHEN att.arc_transaction_type_cd IN ('R','V','E') THEN -a.FARE_AMT ELSE a.FARE_AMT END) AS rev
  from
    subject_revenue.ds_revenue_dw.arc_dds_ticket_coupon_fact a
      join subject_revenue.ds_revenue_dw.arc_transaction_type_dim att
        on a.arc_transaction_type_key = att.arc_transaction_type_key
      join subject_revenue.ds_revenue_dw.market_dim m
        on a.market_key = m.market_key
      join subject_revenue.ds_revenue_dw.carrier_dim oc
        on a.oper_carr_key = oc.carrier_key
      join subject_revenue.ds_revenue_dw.carrier_dim mc
        on a.mktg_carr_key = mc.carrier_key
  where
    a.schd_dptr_dt >= '2023-01-01'
    and a.schd_dptr_dt <= date_add(current_date(), 366)
    and m.od in (select od from aag_ha_od_list) -- only AAG / HA markets
    and CASE
      WHEN oc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD = 'HA' THEN 'HA'
      WHEN mc.ALN_CD IN ('AS', 'QX', 'VX', 'HA') AND oc.ALN_CD IN ('OO') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('AS', 'QX', 'VX') THEN 'AS'
      WHEN oc.ALN_CD IS NULL AND mc.ALN_CD IN ('HA') THEN 'HA'
      WHEN oc.ALN_CD is null and mc.ALN_CD is null THEN NULL
    ELSE 'OA' END = 'OA'
    and date_diff(a.schd_dptr_dt, a.source_file_dt) < 330
    and att.arc_transaction_type_cd != 'E' -- robby suggested removing exchanged transactions
  group by all

In [0]:
create or replace temp view arc_snap_dt as
select
  s.snapshot_date,
  s.dptr_month,
  s.od,
  a.od as arc_od,
  coalesce(a.RPM, 0) as rpm,
  coalesce(a.PAX, 0) as bkg,
  coalesce(a.REV, 0) as rev
from arc_agg a
full outer join snap_shell s
  on a.snap_dt = s.snapshot_date
  and a.dptr_month = s.dptr_month
  and a.od = s.od

In [0]:
create or replace temp view arc_cumulative as
select
  s.*,
  sum(rpm) over (
      partition by dptr_month, od
      order by snapshot_date
    ) as rpm_runttl,
  sum(bkg) over (
      partition by dptr_month, od
      order by snapshot_date
    ) as bkg_runttl,
  sum(rev) over (
      partition by dptr_month, od
      order by snapshot_date
    ) as rev_runttl
from arc_snap_dt s

In [0]:
create or replace temp view ind_cap as
select
  dptr_month
  ,od
  ,sum(asm) as asm
from business_revenuemanagement_test.an_revenuemanagement_ods.rfm_schd_fut_hist_mth
where
  carr_group = 'OA'
  and od in (select od from aag_ha_od_list) -- only AAG / HA markets
group by all

In [0]:
create or replace temp view arc_schd as
select
  c.snapshot_date
  ,c.dptr_month
  ,c.od
  ,c.arc_od
  ,c.rpm as daily_rpm
  ,c.bkg as daily_bkg
  ,c.rev as daily_rev
  ,c.rpm_runttl as rpm
  ,c.bkg_runttl as bkg
  ,c.rev_runttl as rev
  ,coalesce(s.asm, 0) as asm -- industry capacity
from arc_cumulative c
right outer join ind_cap s -- right join to only include legs from cirium
  on s.dptr_month = c.dptr_month
  and s.od = c.od
where
  c.od in (select od from aag_ha_od_list) -- only AAG / HA markets

In [0]:
create or replace temp view empty_od as
select
  concat(od, month(dptr_month)) as od_mth,
  sum(bkg),
  sum(asm)
from arc_schd
group by concat(od, month(dptr_month))
having sum(bkg) = 0 and sum(asm) = 0

In [0]:
create or replace temp view arc_clean as
select *
from arc_schd
where
  concat(od, month(dptr_month)) not in (select od_mth from empty_od)

In [0]:
create or replace temp view arc_cypy as
SELECT
  snapshot_date,
  dptr_month,
  -- last_year_mmdd, -- only used for fixing leap year dates
  od,
  cy_dptr_month,
  cy_rpm,
  cy_bkg,
  cy_rev,
  cy_asm,
  py_dptr_month,
  py_rpm,
  py_bkg,
  py_rev,
  py_asm
FROM
  (
    SELECT
      m.cypy,
      ADD_MONTHS(s.snapshot_date, m.offset) AS snapshot_date,
      m.date_day AS dptr_month,
      -- DATE_FORMAT(date_day, 'MMdd') AS last_year_mmdd, -- only used for fixing leap year dates
      s.od,
      m.day_last_year,
      s.rpm,
      s.bkg,
      s.rev,
      s.asm
    FROM
      arc_clean s
        JOIN cypy_date_map m
          ON m.day_last_year = s.dptr_month
    WHERE m.cypy in ('CY', 'PY')
  )
    PIVOT (
      MAX(day_last_year) AS dptr_month,
      SUM(rpm) AS rpm,
      SUM(bkg) AS bkg,
      SUM(rev) AS rev,
      SUM(asm) AS asm
      FOR cypy IN ('CY' cy, 'PY' py)
    )

In [0]:
create or replace temp view arc_cypy_clean as
SELECT
  r.snapshot_date,
  r.dptr_month,
  r.od,
  r.region,
  r.orig_lat,
  r.orig_long,
  r.dest_lat,
  r.dest_long,
  r.miles,
  COALESCE(r.cy_dptr_month, r.dptr_month) AS cy_dptr_month,
  r.cy_rpm,
  r.cy_bkg,
  r.cy_rev,
  r.cy_asm,
  coalesce(r.py_dptr_month, ADD_MONTHS(r.dptr_month, -12)) as py_dptr_month,
  r.py_rpm,
  r.py_bkg,
  r.py_rev,
  r.py_asm
FROM
  (
    SELECT
      s.snapshot_date,
      s.dptr_month,
      s.od,
      r.region,
      orig_apt.latitude as orig_lat,
      orig_apt.longitude as orig_long,
      dest_apt.latitude as dest_lat,
      dest_apt.longitude as dest_long,
      m.miles,
      MAX(s.cy_dptr_month) AS cy_dptr_month,
      COALESCE(SUM(s.cy_rpm), 0) AS cy_rpm,
      COALESCE(SUM(s.cy_bkg), 0) AS cy_bkg,
      COALESCE(SUM(s.cy_rev), 0) AS cy_rev,
      COALESCE(SUM(s.cy_asm), 0) AS cy_asm,
      MAX(s.py_dptr_month) AS py_dptr_month,
      COALESCE(SUM(s.py_rpm), 0) AS py_rpm,
      COALESCE(SUM(s.py_bkg), 0) AS py_bkg,
      COALESCE(SUM(s.py_rev), 0) AS py_rev,
      COALESCE(SUM(s.py_asm), 0) AS py_asm
    FROM
      arc_cypy s
        join subject_dimensions.ds_dimensions.market_dim m
          on s.od = m.od
        left join business_revenuemanagement_qa.an_revenuemanagement_ods.rfm_all_region_assignment r
          on m.ndod = r.ndod
        join subject_dimensions.ds_dimensions.airport_dim orig_apt
          on m.orig = orig_apt.airport_code
        join subject_dimensions.ds_dimensions.airport_dim dest_apt
          on m.dest = dest_apt.airport_code
    GROUP BY ALL
  ) r

In [0]:
create or replace table cicd_oa_market_data_test as
select *
from arc_cypy_clean